# Bread Shops In New York City Analysis

In [53]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import math

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [58]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [59]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [60]:
neighborhoods_data = newyork_data['features']

In [61]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [62]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [64]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [65]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [66]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [67]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [68]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [69]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [71]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [73]:
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

In [74]:
CLIENT_ID = 'ARPGKVYQLOCOUTBLWIVCB5VOQK11SHF4NPTEW4EAG05ZVAYG' 
CLIENT_SECRET = 'KS2YUBSGDOXWP0PQEHQF4SZCOJVQIKZ2CMDMWCDHZO21C5OD' 
VERSION = '20200220' 

print('Robert Sharkey')
print('ARPGKVYQLOCOUTBLWIVCB5VOQK11SHF4NPTEW4EAG05ZVAYG ' + CLIENT_ID)
print('KS2YUBSGDOXWP0PQEHQF4SZCOJVQIKZ2CMDMWCDHZO21C5OD' + CLIENT_SECRET)

Robert Sharkey
ARPGKVYQLOCOUTBLWIVCB5VOQK11SHF4NPTEW4EAG05ZVAYG ARPGKVYQLOCOUTBLWIVCB5VOQK11SHF4NPTEW4EAG05ZVAYG
KS2YUBSGDOXWP0PQEHQF4SZCOJVQIKZ2CMDMWCDHZO21C5ODKS2YUBSGDOXWP0PQEHQF4SZCOJVQIKZ2CMDMWCDHZO21C5OD


In [78]:
neighborhoods.loc[2, 'Neighborhood']

'Eastchester'

In [79]:
neighborhood_latitude = neighborhoods.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Eastchester are 40.887555677350775, -73.82780644716412.


In [80]:
LIMIT = 200
radius = 2000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ARPGKVYQLOCOUTBLWIVCB5VOQK11SHF4NPTEW4EAG05ZVAYG&client_secret=KS2YUBSGDOXWP0PQEHQF4SZCOJVQIKZ2CMDMWCDHZO21C5OD&v=20200220&ll=40.887555677350775,-73.82780644716412&radius=2000&limit=200&section=food'

In [81]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60307d06c769a12f9a51e075'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Eastchester',
  'headerFullLocation': 'Eastchester, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 85,
  'suggestedBounds': {'ne': {'lat': 40.90555569535079,
    'lng': -73.80404121818248},
   'sw': {'lat': 40.86955565935076, 'lng': -73.85157167614577}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '515cc20ce4b0deb133b8e89b',
       'name': 'Fish & Ting',
       'location': {'address': '4063 Boston Rd',
        'crossStreet': 'Conner St',
        'lat': 40.885656,
        'lng': -73.82

In [82]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [83]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Fish & Ting,Caribbean Restaurant,40.885656,-73.829197
1,Cozy Cottage Restaurant,Diner,40.886332,-73.827616
2,Taco Bell,Fast Food Restaurant,40.883029,-73.824901
3,Gourmet Express,Café,40.890508,-73.817591
4,Panera Bread,Bakery,40.891494,-73.819008


In [84]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

85 venues were returned by Foursquare.


In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            200)
            
        
        print('groups' in requests.get(url).json()["response"])
        if 'groups' in requests.get(url).json()["response"]:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        else:
            continue
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [86]:

ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
True
Co-op City
True
Eastchester
True
Fieldston
True
Riverdale
True
Kingsbridge
True
Marble Hill
True
Woodlawn
True
Norwood
True
Williamsbridge
True
Baychester
True
Pelham Parkway
True
City Island
True
Bedford Park
True
University Heights
True
Morris Heights
True
Fordham
True
East Tremont
True
West Farms
True
High  Bridge
True
Melrose
True
Mott Haven
True
Port Morris
True
Longwood
True
Hunts Point
True
Morrisania
True
Soundview
True
Clason Point
True
Throgs Neck
True
Country Club
True
Parkchester
True
Westchester Square
True
Van Nest
True
Morris Park
True
Belmont
True
Spuyten Duyvil
True
North Riverdale
True
Pelham Bay
True
Schuylerville
True
Edgewater Park
True
Castle Hill
True
Olinville
True
Pelham Gardens
True
Concourse
True
Unionport
True
Edenwald
True
Bay Ridge
True
Bensonhurst
True
Sunset Park
True
Greenpoint
True
Gravesend
True
Brighton Beach
True
Sheepshead Bay
True
Manhattan Terrace
True
Flatbush
True
Crown Heights
True
East Flatbush
True
Kensington
True
Windsor Terr

In [88]:
print(ny_venues.shape)
ny_venues.head()

(26251, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
1,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Kingston Tropical Bakery,40.888568,-73.859885,Bakery
3,Wakefield,40.894705,-73.847201,Jimbo's,40.891740,-73.858226,Burger Joint
4,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant


In [89]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,100,100,100,100,100,100
Annadale,57,57,57,57,57,57
Arden Heights,76,76,76,76,76,76
Arlington,39,39,39,39,39,39
Arrochar,58,58,58,58,58,58
Arverne,31,31,31,31,31,31
Astoria,100,100,100,100,100,100
Astoria Heights,100,100,100,100,100,100
Auburndale,100,100,100,100,100,100


In [90]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 135 uniques categories.


In [91]:
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Churrascaria,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Kofte Place,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint,Yemeni Restaurant
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [92]:
ny_onehot.shape

(26251, 136)

In [93]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Churrascaria,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Kofte Place,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint,Yemeni Restaurant
0,Allerton,0.0,0.0,0.000000,0.01,0.0,0.010000,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.020000,0.0,0.020000,0.0,0.0,0.0,0.010000,0.0,0.0,0.09,0.0,0.040000,0.0,0.0,0.000000,0.0,0.0,0.0,0.040000,0.0,0.020000,0.040000,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.060000,0.000000,0.0,0.020000,0.0,0.0,0.030000,0.0,0.03,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.030000,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.230000,0.0,0.000000,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.120000,0.0,0.01,0.0,0.0,0.000000,0.0,0.01,0.0,0.0,0.0,0.0,0.020000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,Annadale,0.0,0.0,0.052632,0.00,0.0,0.017544,0.0,0.0,0.0,0.070175,0.052632,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.035088,0.0,0.0,0.017544,0.0,0.0,0.0,0.087719,0.0,0.052632,0.017544,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.035088,0.000000,0.0,0.017544,0.0,0.0,0.000000,0.0,0.00,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070175,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.228070,0.0,0.000000,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.052632,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.087719,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Arden Heights,0.0,0.0,0.052632,0.00,0.0,0.052632,0.0,0.0,0.0,0.039474,0.039474,0.0,0.0,0.000000,0.0,0.0

In [94]:
ny_grouped.shape

(302, 136)

In [95]:
num_top_venues = 5
i = 0
for hood in ny_grouped['Neighborhood']:
    if i==5:
        break;
    i=i+1
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                  venue  freq
0           Pizza Place  0.23
1        Sandwich Place  0.12
2  Caribbean Restaurant  0.09
3  Fast Food Restaurant  0.06
4    Italian Restaurant  0.05


----Annadale----
                venue  freq
0         Pizza Place  0.23
1    Sushi Restaurant  0.09
2       Deli / Bodega  0.09
3          Bagel Shop  0.07
4  Italian Restaurant  0.07


----Arden Heights----
                 venue  freq
0          Pizza Place  0.20
1   Italian Restaurant  0.09
2        Deli / Bodega  0.08
3     Sushi Restaurant  0.07
4  American Restaurant  0.05


----Arlington----
                  venue  freq
0         Deli / Bodega  0.23
1  Fast Food Restaurant  0.10
2   American Restaurant  0.05
3           Wings Joint  0.05
4            Donut Shop  0.05


----Arrochar----
                venue  freq
0  Italian Restaurant  0.17
1       Deli / Bodega  0.16
2         Pizza Place  0.14
3          Bagel Shop  0.07
4  Chinese Restaurant  0.07




In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Sandwich Place,Caribbean Restaurant,Fast Food Restaurant,Italian Restaurant,Donut Shop,Deli / Bodega,Chinese Restaurant,Bakery,Food Truck
1,Annadale,Pizza Place,Sushi Restaurant,Deli / Bodega,Bagel Shop,Italian Restaurant,American Restaurant,Sandwich Place,Bakery,Diner,Restaurant
2,Arden Heights,Pizza Place,Italian Restaurant,Deli / Bodega,Sushi Restaurant,American Restaurant,Asian Restaurant,Chinese Restaurant,Fast Food Restaurant,Bakery,Donut Shop
3,Arlington,Deli / Bodega,Fast Food Restaurant,American Restaurant,Wings Joint,Donut Shop,Bagel Shop,Pizza Place,Spanish Restaurant,Chinese Restaurant,Italian Restaurant
4,Arrochar,Italian Restaurant,Deli / Bodega,Pizza Place,Bagel Shop,Chinese Restaurant,Bakery,Food Truck,Restaurant,Sandwich Place,Food


In [98]:
ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

In [99]:
for col in ny_grouped_clustering.columns: 
    print(col)

Afghan Restaurant
African Restaurant
American Restaurant
Arepa Restaurant
Argentinian Restaurant
Asian Restaurant
Australian Restaurant
Austrian Restaurant
BBQ Joint
Bagel Shop
Bakery
Bistro
Brazilian Restaurant
Breakfast Spot
Buffet
Burger Joint
Burmese Restaurant
Burrito Place
Cafeteria
Café
Cajun / Creole Restaurant
Cantonese Restaurant
Caribbean Restaurant
Caucasian Restaurant
Chinese Restaurant
Churrascaria
Colombian Restaurant
Comfort Food Restaurant
Creperie
Cuban Restaurant
Czech Restaurant
Deli / Bodega
Dim Sum Restaurant
Diner
Donut Shop
Dosa Place
Dumpling Restaurant
Eastern European Restaurant
Empanada Restaurant
English Restaurant
Ethiopian Restaurant
Falafel Restaurant
Fast Food Restaurant
Filipino Restaurant
Fish & Chips Shop
Food
Food Court
Food Stand
Food Truck
French Restaurant
Fried Chicken Joint
Gastropub
German Restaurant
Gluten-free Restaurant
Greek Restaurant
Halal Restaurant
Hawaiian Restaurant
Himalayan Restaurant
Hot Dog Joint
Hotpot Restaurant
Indian Restaura

In [100]:
ny_allBreadTypes_clustering = ny_grouped_clustering[['Bagel Shop', 'Bakery', 'Creperie', 'Donut Shop']]
ny_allBreadTypes_clustering.head()

,Bagel Shop,Bakery,Creperie,Donut Shop
0,0.000000,0.040000,0.0,0.040000
1,0.070175,0.052632,0.0,0.017544
2,0.039474,0.039474,0.0,0.039474
3,0.051282,0.000000,0.0,0.051282
4,0.068966,0.051724,0.0,0.017241


In [101]:
ny_grouped_clustering['AllSum'] = ny_grouped_clustering['Bagel Shop'] + ny_grouped_clustering['Bakery'] + ny_grouped_clustering['Creperie'] + ny_grouped_clustering['Donut Shop']
ny_alltypessum_cluster = ny_grouped_clustering[['AllSum']]
ny_alltypessum_cluster.head()

,AllSum
0,0.080000
1,0.140351
2,0.118421
3,0.102564
4,0.137931


In [102]:
ny_Breadrest_clustering_3 = ny_grouped_clustering[['Bagel Shop']]
ny_Breadrest_clustering_3.head()

,Bagel Shop
0,0.000000
1,0.070175
2,0.039474
3,0.051282
4,0.068966


In [103]:
ny_breadrest_clustering_5 = ny_grouped_clustering[['Bagel Shop']]
ny_breadrest_clustering_5.head()

,Bagel Shop
0,0.000000
1,0.070175
2,0.039474
3,0.051282
4,0.068966


In [123]:

kclusters = 5
kmeans_allTypesBreadRest = KMeans(n_clusters=5, random_state=0).fit(ny_alltypessum_cluster)
kmeans_allBreadRest3 = KMeans(n_clusters=3, random_state=0).fit(ny_Breadrest_clustering_3)
kmeans_allBreadRest5 = KMeans(n_clusters=5, random_state=0).fit(ny_breadrest_clustering_5)
kmeans_allrestSum = KMeans(n_clusters=5, random_state=0).fit(ny_alltypessum_cluster)
print(kmeans_allTypesBreadRest.labels_[0:50])
print(kmeans_allBreadRest3.labels_[0:50])
print(kmeans_allBreadRest5.labels_[0:50])
print(kmeans_allrestSum.labels_[0:50])

[4 2 2 4 2 3 2 2 4 1 0 3 3 2 2 0 0 4 2 0 3 4 4 1 2 4 2 2 2 0 4 2 3 0 0 3 4
 4 4 3 2 0 0 4 2 0 2 0 1 4]
[0 1 2 2 1 0 2 2 2 1 0 1 1 2 0 0 0 0 2 0 2 2 0 1 2 0 1 2 0 0 0 2 1 0 0 1 0
 0 1 0 0 0 0 2 2 0 1 0 2 2]
[1 3 0 0 3 1 0 0 4 3 4 3 3 4 4 1 1 4 4 1 0 4 1 3 0 1 2 0 1 1 4 0 3 1 1 3 1
 1 3 4 4 1 4 0 0 1 3 1 0 4]
[4 2 2 4 2 3 2 2 4 1 0 3 3 2 2 0 0 4 2 0 3 4 4 1 2 4 2 2 2 0 4 2 3 0 0 3 4
 4 4 3 2 0 0 4 2 0 2 0 1 4]


In [124]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Sandwich Place,Caribbean Restaurant,Fast Food Restaurant,Italian Restaurant,Donut Shop,Deli / Bodega,Chinese Restaurant,Bakery,Food Truck
1,Annadale,Pizza Place,Sushi Restaurant,Deli / Bodega,Bagel Shop,Italian Restaurant,American Restaurant,Sandwich Place,Bakery,Diner,Restaurant
2,Arden Heights,Pizza Place,Italian Restaurant,Deli / Bodega,Sushi Restaurant,American Restaurant,Asian Restaurant,Chinese Restaurant,Fast Food Restaurant,Bakery,Donut Shop
3,Arlington,Deli / Bodega,Fast Food Restaurant,American Restaurant,Wings Joint,Donut Shop,Bagel Shop,Pizza Place,Spanish Restaurant,Chinese Restaurant,Italian Restaurant
4,Arrochar,Italian Restaurant,Deli / Bodega,Pizza Place,Bagel Shop,Chinese Restaurant,Bakery,Food Truck,Restaurant,Sandwich Place,Food


In [141]:
neighborhoods_venues_sorted.insert(0, 'Cluster_AllBreadTypes2', kmeans_allTypesBreadRest.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster_AllBreadTypes4', kmeans_allBreadRest3.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster_AllBreadTypes6', kmeans_allBreadRest5.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster_AllAddition1', kmeans_allrestSum.labels_)
ny_merged = neighborhoods
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster_AllAddition1,Cluster_AllBreadTypes6,Cluster_AllBreadTypes4,Cluster_AllBreadTypes2,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,1,0,4,1,0,4,4,1,0,4,4,1,0,4,Caribbean Restaurant,Deli / Bodega,Pizza Place,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Bakery,Sandwich Place,Fried Chicken Joint,Donut Shop
1,Bronx,Co-op City,40.874294,-73.829939,4,4,0,4,4,0,4,4,4,0,4,4,4,0,4,Pizza Place,Fast Food Restaurant,Deli / Bodega,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Donut Shop,Spanish Restaurant,Bakery,Fried Chicken Joint
2,Bronx,Eastchester,40.887556,-73.827806,4,1,0,4,1,0,4,4,1,0,4,4,1,0,4,Fast Food Restaurant,Pizza Place,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,Diner,Café,Burger Joint
3,Bronx,Fieldston,40.895437,-73.905643,2,4,2,2,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Deli / Bodega,Burger Joint,Latin American Restaurant,Chinese Restaurant,Café
4,Bronx,Riverdale,40.890834,-73.912585,2,4,2,2,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Latin American Restaurant,Chinese Restaurant,Deli / Bodega,Bakery,Italian Restaurant


In [142]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
rainbow = ['#8000ff', '#FE0303', '#049A41', '#790272', '#EEFE03']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster_AllTypes']):
    if math.isnan(cluster):
        continue

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [143]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes4'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,1,0,4,4,1,0,4,4,1,0,4,Caribbean Restaurant,Deli / Bodega,Pizza Place,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Bakery,Sandwich Place,Fried Chicken Joint,Donut Shop
1,Co-op City,Bronx,4,0,4,4,4,0,4,4,4,0,4,Pizza Place,Fast Food Restaurant,Deli / Bodega,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Donut Shop,Spanish Restaurant,Bakery,Fried Chicken Joint
2,Eastchester,Bronx,1,0,4,4,1,0,4,4,1,0,4,Fast Food Restaurant,Pizza Place,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,Diner,Café,Burger Joint
5,Kingsbridge,Bronx,4,0,2,2,4,0,2,2,4,0,2,Pizza Place,Mexican Restaurant,Sandwich Place,Diner,Donut Shop,Deli / Bodega,Spanish Restaurant,Bakery,Café,Fast Food Restaurant
6,Marble Hill,Manhattan,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Mexican Restaurant,Diner,Café,Latin American Restaurant,Spanish Restaurant,Bakery,Deli / Bodega,Restaurant,Chinese Restaurant
8,Norwood,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Diner,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Chinese Restaurant,Deli / Bodega
9,Williamsbridge,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Spanish Restaurant,Donut Shop,Chinese Restaurant
11,Pelham Parkway,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Chinese Restaurant,Sandwich Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Fast Food Restaurant,Donut Shop,Caribbean Restaurant,Food Truck
12,City Island,Bronx,1,0,0,0,1,0,0,0,1,0,0,Seafood Restaurant,Deli / Bodega,Italian Restaurant,Café,Sandwich Place,Donut Shop,Spanish Restaurant,Pizza Place,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,1,0,0,0,1,0,0,0,1,0,0,Italian Restaurant,Pizza Place,Mexican Restaurant,Diner,Deli / Bodega,Caribbean Restaurant,Spanish Restaurant,Sandwich Place,Bakery,Latin American Restaurant


In [144]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes4'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Bay Ridge,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Mexican Restaurant,Greek Restaurant,Japanese Restaurant,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant
47,Bensonhurst,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Bakery,Italian Restaurant,Pizza Place,Chinese Restaurant,Sushi Restaurant,Bagel Shop,Sandwich Place,Diner,Japanese Restaurant,Cantonese Restaurant
50,Gravesend,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Sushi Restaurant,Bakery,Pizza Place,Bagel Shop,Sandwich Place,Vietnamese Restaurant,Italian Restaurant,Donut Shop,Chinese Restaurant,Restaurant
53,Manhattan Terrace,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Sushi Restaurant,Bagel Shop,Bakery,Donut Shop,Café,Sandwich Place,Italian Restaurant,Steakhouse,Japanese Restaurant
58,Windsor Terrace,Brooklyn,3,1,2,2,3,1,2,2,3,1,2,Mexican Restaurant,Café,Italian Restaurant,Pizza Place,Bakery,Bagel Shop,American Restaurant,Thai Restaurant,Deli / Bodega,Caribbean Restaurant
64,Brooklyn Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Pizza Place,Bagel Shop,Italian Restaurant,Deli / Bodega,Japanese Restaurant,Café,Burger Joint,American Restaurant,Sandwich Place
70,Park Slope,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Italian Restaurant,Mexican Restaurant,Bagel Shop,Pizza Place,Sushi Restaurant,Café,Vietnamese Restaurant,New American Restaurant,American Restaurant
76,Mill Island,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Chinese Restaurant,Italian Restaurant,Bagel Shop,Deli / Bodega,Japanese Restaurant,Donut Shop,Bakery,Sandwich Place,Hot Dog Joint
79,Bath Beach,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Chinese Restaurant,Sushi Restaurant,Donut Shop,Diner,Dim Sum Restaurant,Cantonese Restaurant
81,Dyker Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Greek Restaurant,Chinese Restaurant,Japanese Restaurant,Middle Eastern Restaurant,American Restaurant,Mexican Restaurant


In [145]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes4'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Deli / Bodega,Burger Joint,Latin American Restaurant,Chinese Restaurant,Café
4,Riverdale,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Latin American Restaurant,Chinese Restaurant,Deli / Bodega,Bakery,Italian Restaurant
7,Woodlawn,Bronx,4,2,2,2,4,2,2,2,4,2,2,Caribbean Restaurant,Pizza Place,Deli / Bodega,American Restaurant,Sandwich Place,Donut Shop,Italian Restaurant,Diner,Bakery,Fast Food Restaurant
10,Baychester,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,Bakery,Italian Restaurant,Bagel Shop
42,Pelham Gardens,Bronx,4,2,3,3,4,2,3,3,4,2,3,Pizza Place,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Bakery,Donut Shop,Caribbean Restaurant,Deli / Bodega,Mexican Restaurant,Chinese Restaurant
49,Greenpoint,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Café,Pizza Place,Italian Restaurant,Bakery,Mexican Restaurant,Japanese Restaurant,Restaurant,Bagel Shop,Polish Restaurant,Deli / Bodega
51,Brighton Beach,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Pizza Place,Sushi Restaurant,Restaurant,Eastern European Restaurant,Bakery,Russian Restaurant,Italian Restaurant,Bagel Shop,Seafood Restaurant,Japanese Restaurant
52,Sheepshead Bay,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Sushi Restaurant,Pizza Place,Italian Restaurant,Sandwich Place,Seafood Restaurant,Restaurant,Japanese Restaurant,Russian Restaurant,Bagel Shop,Eastern European Restaurant
54,Flatbush,Brooklyn,4,2,4,4,4,2,4,4,4,2,4,Caribbean Restaurant,Pizza Place,Mexican Restaurant,Café,Thai Restaurant,Deli / Bodega,Bakery,Chinese Restaurant,Latin American Restaurant,Bagel Shop
55,Crown Heights,Brooklyn,4,2,4,4,4,2,4,4,4,2,4,Café,Pizza Place,Caribbean Restaurant,Bakery,Italian Restaurant,Sushi Restaurant,Southern / Soul Food Restaurant,Deli / Bodega,Bagel Shop,Thai Restaurant


In [146]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes6'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Greenpoint,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Café,Pizza Place,Italian Restaurant,Bakery,Mexican Restaurant,Japanese Restaurant,Restaurant,Bagel Shop,Polish Restaurant,Deli / Bodega
51,Brighton Beach,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Pizza Place,Sushi Restaurant,Restaurant,Eastern European Restaurant,Bakery,Russian Restaurant,Italian Restaurant,Bagel Shop,Seafood Restaurant,Japanese Restaurant
52,Sheepshead Bay,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Sushi Restaurant,Pizza Place,Italian Restaurant,Sandwich Place,Seafood Restaurant,Restaurant,Japanese Restaurant,Russian Restaurant,Bagel Shop,Eastern European Restaurant
59,Prospect Heights,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Bakery,Pizza Place,New American Restaurant,Sushi Restaurant,Mexican Restaurant,American Restaurant,Vietnamese Restaurant,Bagel Shop,Thai Restaurant,Café
65,Cobble Hill,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Italian Restaurant,Bakery,Pizza Place,French Restaurant,Deli / Bodega,Japanese Restaurant,Thai Restaurant,Bagel Shop,Sushi Restaurant,American Restaurant
66,Carroll Gardens,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Italian Restaurant,Bakery,Pizza Place,French Restaurant,American Restaurant,Seafood Restaurant,Deli / Bodega,Bagel Shop,Thai Restaurant,Japanese Restaurant
68,Gowanus,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Italian Restaurant,Pizza Place,Bakery,American Restaurant,Mexican Restaurant,Bagel Shop,Deli / Bodega,Seafood Restaurant,Food Truck,Japanese Restaurant
73,Starrett City,Brooklyn,0,2,4,4,0,2,4,4,0,2,4,Pizza Place,Fast Food Restaurant,Caribbean Restaurant,Chinese Restaurant,Fried Chicken Joint,Breakfast Spot,Food,Donut Shop,Sandwich Place,Bagel Shop
75,Flatlands,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Caribbean Restaurant,Pizza Place,Sandwich Place,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Deli / Bodega,Donut Shop,Bagel Shop,Restaurant
78,Coney Island,Brooklyn,0,2,4,4,0,2,4,4,0,2,4,Pizza Place,Deli / Bodega,Restaurant,American Restaurant,Hot Dog Joint,Mexican Restaurant,Sushi Restaurant,Fast Food Restaurant,Bagel Shop,Caribbean Restaurant


In [147]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes6'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,1,0,4,4,1,0,4,4,1,0,4,Caribbean Restaurant,Deli / Bodega,Pizza Place,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Bakery,Sandwich Place,Fried Chicken Joint,Donut Shop
2,Eastchester,Bronx,1,0,4,4,1,0,4,4,1,0,4,Fast Food Restaurant,Pizza Place,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,Diner,Café,Burger Joint
6,Marble Hill,Manhattan,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Mexican Restaurant,Diner,Café,Latin American Restaurant,Spanish Restaurant,Bakery,Deli / Bodega,Restaurant,Chinese Restaurant
8,Norwood,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Diner,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Chinese Restaurant,Deli / Bodega
9,Williamsbridge,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Spanish Restaurant,Donut Shop,Chinese Restaurant
11,Pelham Parkway,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Chinese Restaurant,Sandwich Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Fast Food Restaurant,Donut Shop,Caribbean Restaurant,Food Truck
12,City Island,Bronx,1,0,0,0,1,0,0,0,1,0,0,Seafood Restaurant,Deli / Bodega,Italian Restaurant,Café,Sandwich Place,Donut Shop,Spanish Restaurant,Pizza Place,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,1,0,0,0,1,0,0,0,1,0,0,Italian Restaurant,Pizza Place,Mexican Restaurant,Diner,Deli / Bodega,Caribbean Restaurant,Spanish Restaurant,Sandwich Place,Bakery,Latin American Restaurant
14,University Heights,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Italian Restaurant,Mexican Restaurant,Latin American Restaurant,Deli / Bodega,Bakery,Spanish Restaurant,Café,Diner,Restaurant
15,Morris Heights,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Latin American Restaurant,Spanish Restaurant,Deli / Bodega,Bakery,Mexican Restaurant,Fast Food Restaurant,Café,Tapas Restaurant,Restaurant


In [148]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes6'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Marine Park,Brooklyn,2,1,3,3,2,1,3,3,2,1,3,Pizza Place,Bagel Shop,Italian Restaurant,Chinese Restaurant,Sandwich Place,Bakery,Sushi Restaurant,Japanese Restaurant,Deli / Bodega,Diner
203,Todt Hill,Staten Island,2,1,3,3,2,1,3,3,2,1,3,Pizza Place,Deli / Bodega,Bagel Shop,Italian Restaurant,Chinese Restaurant,Donut Shop,Sandwich Place,Japanese Restaurant,American Restaurant,Sushi Restaurant
214,Eltingville,Staten Island,2,1,3,3,2,1,3,3,2,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Sushi Restaurant,Diner,Deli / Bodega,Sandwich Place,Bakery,Fast Food Restaurant,Chinese Restaurant
216,Woodrow,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Donut Shop,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Sushi Restaurant,Food,Sandwich Place,Deli / Bodega
240,Rossville,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Chinese Restaurant,Donut Shop,Mexican Restaurant,Italian Restaurant,Food,Sushi Restaurant,Bakery,Diner
245,Bloomfield,Staten Island,2,1,2,2,2,1,2,2,2,1,2,Deli / Bodega,Italian Restaurant,Pizza Place,Food,Spanish Restaurant,Bagel Shop,Steakhouse,Sandwich Place,Café,Burger Joint
286,Sandy Ground,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Donut Shop,Bagel Shop,Bakery,Chinese Restaurant,Deli / Bodega,Diner,Italian Restaurant,Sushi Restaurant,German Restaurant
292,Lighthouse Hill,Staten Island,2,1,3,3,2,1,3,3,2,1,3,Italian Restaurant,Deli / Bodega,Bagel Shop,Mexican Restaurant,Pizza Place,Sandwich Place,Bakery,Asian Restaurant,Restaurant,Food
296,Madison,Brooklyn,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Sushi Restaurant,Bakery,Italian Restaurant,Sandwich Place,Japanese Restaurant,Donut Shop,Diner,Russian Restaurant


In [149]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes6'] == 3, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Bay Ridge,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Mexican Restaurant,Greek Restaurant,Japanese Restaurant,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant
47,Bensonhurst,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Bakery,Italian Restaurant,Pizza Place,Chinese Restaurant,Sushi Restaurant,Bagel Shop,Sandwich Place,Diner,Japanese Restaurant,Cantonese Restaurant
50,Gravesend,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Sushi Restaurant,Bakery,Pizza Place,Bagel Shop,Sandwich Place,Vietnamese Restaurant,Italian Restaurant,Donut Shop,Chinese Restaurant,Restaurant
53,Manhattan Terrace,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Sushi Restaurant,Bagel Shop,Bakery,Donut Shop,Café,Sandwich Place,Italian Restaurant,Steakhouse,Japanese Restaurant
58,Windsor Terrace,Brooklyn,3,1,2,2,3,1,2,2,3,1,2,Mexican Restaurant,Café,Italian Restaurant,Pizza Place,Bakery,Bagel Shop,American Restaurant,Thai Restaurant,Deli / Bodega,Caribbean Restaurant
64,Brooklyn Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Pizza Place,Bagel Shop,Italian Restaurant,Deli / Bodega,Japanese Restaurant,Café,Burger Joint,American Restaurant,Sandwich Place
70,Park Slope,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Italian Restaurant,Mexican Restaurant,Bagel Shop,Pizza Place,Sushi Restaurant,Café,Vietnamese Restaurant,New American Restaurant,American Restaurant
76,Mill Island,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Chinese Restaurant,Italian Restaurant,Bagel Shop,Deli / Bodega,Japanese Restaurant,Donut Shop,Bakery,Sandwich Place,Hot Dog Joint
79,Bath Beach,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Chinese Restaurant,Sushi Restaurant,Donut Shop,Diner,Dim Sum Restaurant,Cantonese Restaurant
81,Dyker Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Greek Restaurant,Chinese Restaurant,Japanese Restaurant,Middle Eastern Restaurant,American Restaurant,Mexican Restaurant


In [150]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes6'] == 4, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bronx,4,0,4,4,4,0,4,4,4,0,4,Pizza Place,Fast Food Restaurant,Deli / Bodega,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Donut Shop,Spanish Restaurant,Bakery,Fried Chicken Joint
3,Fieldston,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Deli / Bodega,Burger Joint,Latin American Restaurant,Chinese Restaurant,Café
4,Riverdale,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Latin American Restaurant,Chinese Restaurant,Deli / Bodega,Bakery,Italian Restaurant
5,Kingsbridge,Bronx,4,0,2,2,4,0,2,2,4,0,2,Pizza Place,Mexican Restaurant,Sandwich Place,Diner,Donut Shop,Deli / Bodega,Spanish Restaurant,Bakery,Café,Fast Food Restaurant
7,Woodlawn,Bronx,4,2,2,2,4,2,2,2,4,2,2,Caribbean Restaurant,Pizza Place,Deli / Bodega,American Restaurant,Sandwich Place,Donut Shop,Italian Restaurant,Diner,Bakery,Fast Food Restaurant
10,Baychester,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,Bakery,Italian Restaurant,Bagel Shop
35,Spuyten Duyvil,Bronx,4,0,4,4,4,0,4,4,4,0,4,Mexican Restaurant,Pizza Place,Sandwich Place,Café,Deli / Bodega,Restaurant,Spanish Restaurant,Latin American Restaurant,Diner,Donut Shop
36,North Riverdale,Bronx,4,0,2,2,4,0,2,2,4,0,2,Pizza Place,Deli / Bodega,Sandwich Place,Italian Restaurant,Chinese Restaurant,Donut Shop,Food,Restaurant,Mexican Restaurant,Bakery
37,Pelham Bay,Bronx,4,0,2,2,4,0,2,2,4,0,2,Sandwich Place,Pizza Place,Italian Restaurant,Bakery,Fast Food Restaurant,Mexican Restaurant,Deli / Bodega,Donut Shop,Diner,Sushi Restaurant
42,Pelham Gardens,Bronx,4,2,3,3,4,2,3,3,4,2,3,Pizza Place,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Bakery,Donut Shop,Caribbean Restaurant,Deli / Bodega,Mexican Restaurant,Chinese Restaurant


In [153]:
ny_merged.loc[ny_merged['Cluster_AllAddition1'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Pelham Parkway,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Chinese Restaurant,Sandwich Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Fast Food Restaurant,Donut Shop,Caribbean Restaurant,Food Truck
12,City Island,Bronx,1,0,0,0,1,0,0,0,1,0,0,Seafood Restaurant,Deli / Bodega,Italian Restaurant,Café,Sandwich Place,Donut Shop,Spanish Restaurant,Pizza Place,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,1,0,0,0,1,0,0,0,1,0,0,Italian Restaurant,Pizza Place,Mexican Restaurant,Diner,Deli / Bodega,Caribbean Restaurant,Spanish Restaurant,Sandwich Place,Bakery,Latin American Restaurant
14,University Heights,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Italian Restaurant,Mexican Restaurant,Latin American Restaurant,Deli / Bodega,Bakery,Spanish Restaurant,Café,Diner,Restaurant
16,Fordham,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Diner,Spanish Restaurant,Latin American Restaurant,Bakery,Sandwich Place,Wings Joint
19,High Bridge,Bronx,1,0,0,0,1,0,0,0,1,0,0,Mexican Restaurant,Deli / Bodega,Pizza Place,Latin American Restaurant,Sandwich Place,Spanish Restaurant,Bakery,Caribbean Restaurant,Burger Joint,Italian Restaurant
21,Mott Haven,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Mexican Restaurant,Deli / Bodega,Diner,Bakery,Fried Chicken Joint,Café,Food Truck,American Restaurant,Latin American Restaurant
22,Port Morris,Bronx,1,0,0,0,1,0,0,0,1,0,0,Mexican Restaurant,Pizza Place,Diner,Café,Latin American Restaurant,Spanish Restaurant,Burger Joint,Peruvian Restaurant,Restaurant,Italian Restaurant
26,Soundview,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Deli / Bodega,Fast Food Restaurant,Fried Chicken Joint,Latin American Restaurant,Donut Shop,Sandwich Place,Mexican Restaurant,Food,Diner
27,Clason Point,Bronx,1,0,0,0,1,0,0,0,1,0,0,Food,Deli / Bodega,Pizza Place,Chinese Restaurant,Latin American Restaurant,Korean Restaurant,Restaurant,Seafood Restaurant,Diner,Food Truck


In [154]:
ny_merged.loc[ny_merged['Cluster_AllAddition1'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Bensonhurst,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Bakery,Italian Restaurant,Pizza Place,Chinese Restaurant,Sushi Restaurant,Bagel Shop,Sandwich Place,Diner,Japanese Restaurant,Cantonese Restaurant
50,Gravesend,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Sushi Restaurant,Bakery,Pizza Place,Bagel Shop,Sandwich Place,Vietnamese Restaurant,Italian Restaurant,Donut Shop,Chinese Restaurant,Restaurant
53,Manhattan Terrace,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Sushi Restaurant,Bagel Shop,Bakery,Donut Shop,Café,Sandwich Place,Italian Restaurant,Steakhouse,Japanese Restaurant
79,Bath Beach,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Chinese Restaurant,Sushi Restaurant,Donut Shop,Diner,Dim Sum Restaurant,Cantonese Restaurant
98,Ocean Parkway,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Bakery,Sushi Restaurant,Bagel Shop,Donut Shop,Italian Restaurant,Café,Japanese Restaurant,Chinese Restaurant,Middle Eastern Restaurant
216,Woodrow,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Donut Shop,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Sushi Restaurant,Food,Sandwich Place,Deli / Bodega
237,Pleasant Plains,Staten Island,0,2,1,1,0,2,1,1,0,2,1,Pizza Place,Fast Food Restaurant,Bakery,Italian Restaurant,Deli / Bodega,Bagel Shop,Donut Shop,American Restaurant,Sushi Restaurant,Food
239,Charleston,Staten Island,0,2,1,1,0,2,1,1,0,2,1,Bakery,Pizza Place,Fast Food Restaurant,American Restaurant,Deli / Bodega,Food,Donut Shop,Italian Restaurant,Breakfast Spot,Bagel Shop
240,Rossville,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Chinese Restaurant,Donut Shop,Mexican Restaurant,Italian Restaurant,Food,Sushi Restaurant,Bakery,Diner
257,Howland Hook,Staten Island,0,2,1,1,0,2,1,1,0,2,1,Food,Bakery,Pizza Place,Latin American Restaurant,Deli / Bodega,Sandwich Place,Restaurant,Donut Shop,Snack Place,Portuguese Restaurant


In [155]:
ny_merged.loc[ny_merged['Cluster_AllAddition1'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Deli / Bodega,Burger Joint,Latin American Restaurant,Chinese Restaurant,Café
4,Riverdale,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Diner,Latin American Restaurant,Chinese Restaurant,Deli / Bodega,Bakery,Italian Restaurant
5,Kingsbridge,Bronx,4,0,2,2,4,0,2,2,4,0,2,Pizza Place,Mexican Restaurant,Sandwich Place,Diner,Donut Shop,Deli / Bodega,Spanish Restaurant,Bakery,Café,Fast Food Restaurant
7,Woodlawn,Bronx,4,2,2,2,4,2,2,2,4,2,2,Caribbean Restaurant,Pizza Place,Deli / Bodega,American Restaurant,Sandwich Place,Donut Shop,Italian Restaurant,Diner,Bakery,Fast Food Restaurant
10,Baychester,Bronx,4,2,2,2,4,2,2,2,4,2,2,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,Bakery,Italian Restaurant,Bagel Shop
17,East Tremont,Bronx,1,0,2,2,1,0,2,2,1,0,2,Pizza Place,Italian Restaurant,Deli / Bodega,Fast Food Restaurant,Bakery,Donut Shop,Sandwich Place,Spanish Restaurant,Diner,Mexican Restaurant
33,Morris Park,Bronx,1,0,2,2,1,0,2,2,1,0,2,Pizza Place,Italian Restaurant,Sandwich Place,Mexican Restaurant,Bakery,Diner,Donut Shop,Deli / Bodega,Chinese Restaurant,Latin American Restaurant
36,North Riverdale,Bronx,4,0,2,2,4,0,2,2,4,0,2,Pizza Place,Deli / Bodega,Sandwich Place,Italian Restaurant,Chinese Restaurant,Donut Shop,Food,Restaurant,Mexican Restaurant,Bakery
37,Pelham Bay,Bronx,4,0,2,2,4,0,2,2,4,0,2,Sandwich Place,Pizza Place,Italian Restaurant,Bakery,Fast Food Restaurant,Mexican Restaurant,Deli / Bodega,Donut Shop,Diner,Sushi Restaurant
49,Greenpoint,Brooklyn,0,2,2,2,0,2,2,2,0,2,2,Café,Pizza Place,Italian Restaurant,Bakery,Mexican Restaurant,Japanese Restaurant,Restaurant,Bagel Shop,Polish Restaurant,Deli / Bodega


In [156]:
ny_merged.loc[ny_merged['Cluster_AllAddition1'] == 3, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Pelham Gardens,Bronx,4,2,3,3,4,2,3,3,4,2,3,Pizza Place,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Bakery,Donut Shop,Caribbean Restaurant,Deli / Bodega,Mexican Restaurant,Chinese Restaurant
46,Bay Ridge,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Mexican Restaurant,Greek Restaurant,Japanese Restaurant,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant
48,Sunset Park,Brooklyn,4,0,3,3,4,0,3,3,4,0,3,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Chinese Restaurant,Café,Restaurant,American Restaurant,Vietnamese Restaurant,Italian Restaurant
62,Bushwick,Brooklyn,4,0,3,3,4,0,3,3,4,0,3,Bakery,Pizza Place,Mexican Restaurant,Deli / Bodega,Café,Italian Restaurant,Taco Place,Donut Shop,Japanese Restaurant,French Restaurant
64,Brooklyn Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Pizza Place,Bagel Shop,Italian Restaurant,Deli / Bodega,Japanese Restaurant,Café,Burger Joint,American Restaurant,Sandwich Place
70,Park Slope,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Italian Restaurant,Mexican Restaurant,Bagel Shop,Pizza Place,Sushi Restaurant,Café,Vietnamese Restaurant,New American Restaurant,American Restaurant
76,Mill Island,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Chinese Restaurant,Italian Restaurant,Bagel Shop,Deli / Bodega,Japanese Restaurant,Donut Shop,Bakery,Sandwich Place,Hot Dog Joint
81,Dyker Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Greek Restaurant,Chinese Restaurant,Japanese Restaurant,Middle Eastern Restaurant,American Restaurant,Mexican Restaurant
83,Marine Park,Brooklyn,2,1,3,3,2,1,3,3,2,1,3,Pizza Place,Bagel Shop,Italian Restaurant,Chinese Restaurant,Sandwich Place,Bakery,Sushi Restaurant,Japanese Restaurant,Deli / Bodega,Diner
99,Fort Hamilton,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Sandwich Place,American Restaurant,Chinese Restaurant,Greek Restaurant,Japanese Restaurant,Diner


In [157]:
ny_merged.loc[ny_merged['Cluster_AllAddition1'] == 4, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,1,0,4,4,1,0,4,4,1,0,4,Caribbean Restaurant,Deli / Bodega,Pizza Place,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Bakery,Sandwich Place,Fried Chicken Joint,Donut Shop
1,Co-op City,Bronx,4,0,4,4,4,0,4,4,4,0,4,Pizza Place,Fast Food Restaurant,Deli / Bodega,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Donut Shop,Spanish Restaurant,Bakery,Fried Chicken Joint
2,Eastchester,Bronx,1,0,4,4,1,0,4,4,1,0,4,Fast Food Restaurant,Pizza Place,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,Diner,Café,Burger Joint
6,Marble Hill,Manhattan,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Mexican Restaurant,Diner,Café,Latin American Restaurant,Spanish Restaurant,Bakery,Deli / Bodega,Restaurant,Chinese Restaurant
8,Norwood,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Diner,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Chinese Restaurant,Deli / Bodega
9,Williamsbridge,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Spanish Restaurant,Donut Shop,Chinese Restaurant
15,Morris Heights,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Latin American Restaurant,Spanish Restaurant,Deli / Bodega,Bakery,Mexican Restaurant,Fast Food Restaurant,Café,Tapas Restaurant,Restaurant
18,West Farms,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Italian Restaurant,Donut Shop,Deli / Bodega,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Breakfast Spot,Diner,Bakery
20,Melrose,Bronx,1,0,4,4,1,0,4,4,1,0,4,Sandwich Place,Pizza Place,Mexican Restaurant,Donut Shop,Diner,Fried Chicken Joint,Deli / Bodega,Caribbean Restaurant,Fast Food Restaurant,Latin American Restaurant
23,Longwood,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Mexican Restaurant,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Donut Shop,Spanish Restaurant,Diner,Latin American Restaurant,Chinese Restaurant


In [159]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes2'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Pelham Parkway,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Chinese Restaurant,Sandwich Place,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Fast Food Restaurant,Donut Shop,Caribbean Restaurant,Food Truck
12,City Island,Bronx,1,0,0,0,1,0,0,0,1,0,0,Seafood Restaurant,Deli / Bodega,Italian Restaurant,Café,Sandwich Place,Donut Shop,Spanish Restaurant,Pizza Place,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,1,0,0,0,1,0,0,0,1,0,0,Italian Restaurant,Pizza Place,Mexican Restaurant,Diner,Deli / Bodega,Caribbean Restaurant,Spanish Restaurant,Sandwich Place,Bakery,Latin American Restaurant
14,University Heights,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Italian Restaurant,Mexican Restaurant,Latin American Restaurant,Deli / Bodega,Bakery,Spanish Restaurant,Café,Diner,Restaurant
16,Fordham,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Italian Restaurant,Mexican Restaurant,Deli / Bodega,Diner,Spanish Restaurant,Latin American Restaurant,Bakery,Sandwich Place,Wings Joint
19,High Bridge,Bronx,1,0,0,0,1,0,0,0,1,0,0,Mexican Restaurant,Deli / Bodega,Pizza Place,Latin American Restaurant,Sandwich Place,Spanish Restaurant,Bakery,Caribbean Restaurant,Burger Joint,Italian Restaurant
21,Mott Haven,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Mexican Restaurant,Deli / Bodega,Diner,Bakery,Fried Chicken Joint,Café,Food Truck,American Restaurant,Latin American Restaurant
22,Port Morris,Bronx,1,0,0,0,1,0,0,0,1,0,0,Mexican Restaurant,Pizza Place,Diner,Café,Latin American Restaurant,Spanish Restaurant,Burger Joint,Peruvian Restaurant,Restaurant,Italian Restaurant
26,Soundview,Bronx,1,0,0,0,1,0,0,0,1,0,0,Pizza Place,Deli / Bodega,Fast Food Restaurant,Fried Chicken Joint,Latin American Restaurant,Donut Shop,Sandwich Place,Mexican Restaurant,Food,Diner
27,Clason Point,Bronx,1,0,0,0,1,0,0,0,1,0,0,Food,Deli / Bodega,Pizza Place,Chinese Restaurant,Latin American Restaurant,Korean Restaurant,Restaurant,Seafood Restaurant,Diner,Food Truck


In [160]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes2'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Bensonhurst,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Bakery,Italian Restaurant,Pizza Place,Chinese Restaurant,Sushi Restaurant,Bagel Shop,Sandwich Place,Diner,Japanese Restaurant,Cantonese Restaurant
50,Gravesend,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Sushi Restaurant,Bakery,Pizza Place,Bagel Shop,Sandwich Place,Vietnamese Restaurant,Italian Restaurant,Donut Shop,Chinese Restaurant,Restaurant
53,Manhattan Terrace,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Sushi Restaurant,Bagel Shop,Bakery,Donut Shop,Café,Sandwich Place,Italian Restaurant,Steakhouse,Japanese Restaurant
79,Bath Beach,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Chinese Restaurant,Sushi Restaurant,Donut Shop,Diner,Dim Sum Restaurant,Cantonese Restaurant
98,Ocean Parkway,Brooklyn,3,1,1,1,3,1,1,1,3,1,1,Pizza Place,Bakery,Sushi Restaurant,Bagel Shop,Donut Shop,Italian Restaurant,Café,Japanese Restaurant,Chinese Restaurant,Middle Eastern Restaurant
216,Woodrow,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Donut Shop,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Sushi Restaurant,Food,Sandwich Place,Deli / Bodega
237,Pleasant Plains,Staten Island,0,2,1,1,0,2,1,1,0,2,1,Pizza Place,Fast Food Restaurant,Bakery,Italian Restaurant,Deli / Bodega,Bagel Shop,Donut Shop,American Restaurant,Sushi Restaurant,Food
239,Charleston,Staten Island,0,2,1,1,0,2,1,1,0,2,1,Bakery,Pizza Place,Fast Food Restaurant,American Restaurant,Deli / Bodega,Food,Donut Shop,Italian Restaurant,Breakfast Spot,Bagel Shop
240,Rossville,Staten Island,2,1,1,1,2,1,1,1,2,1,1,Pizza Place,Bagel Shop,Chinese Restaurant,Donut Shop,Mexican Restaurant,Italian Restaurant,Food,Sushi Restaurant,Bakery,Diner
257,Howland Hook,Staten Island,0,2,1,1,0,2,1,1,0,2,1,Food,Bakery,Pizza Place,Latin American Restaurant,Deli / Bodega,Sandwich Place,Restaurant,Donut Shop,Snack Place,Portuguese Restaurant


In [161]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes2'] == 3, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Pelham Gardens,Bronx,4,2,3,3,4,2,3,3,4,2,3,Pizza Place,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Bakery,Donut Shop,Caribbean Restaurant,Deli / Bodega,Mexican Restaurant,Chinese Restaurant
46,Bay Ridge,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Mexican Restaurant,Greek Restaurant,Japanese Restaurant,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant
48,Sunset Park,Brooklyn,4,0,3,3,4,0,3,3,4,0,3,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Chinese Restaurant,Café,Restaurant,American Restaurant,Vietnamese Restaurant,Italian Restaurant
62,Bushwick,Brooklyn,4,0,3,3,4,0,3,3,4,0,3,Bakery,Pizza Place,Mexican Restaurant,Deli / Bodega,Café,Italian Restaurant,Taco Place,Donut Shop,Japanese Restaurant,French Restaurant
64,Brooklyn Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Pizza Place,Bagel Shop,Italian Restaurant,Deli / Bodega,Japanese Restaurant,Café,Burger Joint,American Restaurant,Sandwich Place
70,Park Slope,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Bakery,Italian Restaurant,Mexican Restaurant,Bagel Shop,Pizza Place,Sushi Restaurant,Café,Vietnamese Restaurant,New American Restaurant,American Restaurant
76,Mill Island,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Chinese Restaurant,Italian Restaurant,Bagel Shop,Deli / Bodega,Japanese Restaurant,Donut Shop,Bakery,Sandwich Place,Hot Dog Joint
81,Dyker Heights,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Greek Restaurant,Chinese Restaurant,Japanese Restaurant,Middle Eastern Restaurant,American Restaurant,Mexican Restaurant
83,Marine Park,Brooklyn,2,1,3,3,2,1,3,3,2,1,3,Pizza Place,Bagel Shop,Italian Restaurant,Chinese Restaurant,Sandwich Place,Bakery,Sushi Restaurant,Japanese Restaurant,Deli / Bodega,Diner
99,Fort Hamilton,Brooklyn,3,1,3,3,3,1,3,3,3,1,3,Pizza Place,Italian Restaurant,Bagel Shop,Bakery,Sandwich Place,American Restaurant,Chinese Restaurant,Greek Restaurant,Japanese Restaurant,Diner


In [162]:
ny_merged.loc[ny_merged['Cluster_AllBreadTypes2'] == 4, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,Cluster_AllBre5,Cluster_AllBre3,Cluster_AllBreadTypes1,Cluster_AllBreTypes,Cluster_AllBread5,Cluster_AllBread3,Cluster_AllBreadTypes,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,1,0,4,4,1,0,4,4,1,0,4,Caribbean Restaurant,Deli / Bodega,Pizza Place,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Bakery,Sandwich Place,Fried Chicken Joint,Donut Shop
1,Co-op City,Bronx,4,0,4,4,4,0,4,4,4,0,4,Pizza Place,Fast Food Restaurant,Deli / Bodega,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Donut Shop,Spanish Restaurant,Bakery,Fried Chicken Joint
2,Eastchester,Bronx,1,0,4,4,1,0,4,4,1,0,4,Fast Food Restaurant,Pizza Place,Deli / Bodega,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,Diner,Café,Burger Joint
6,Marble Hill,Manhattan,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Mexican Restaurant,Diner,Café,Latin American Restaurant,Spanish Restaurant,Bakery,Deli / Bodega,Restaurant,Chinese Restaurant
8,Norwood,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Diner,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Chinese Restaurant,Deli / Bodega
9,Williamsbridge,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Spanish Restaurant,Donut Shop,Chinese Restaurant
15,Morris Heights,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Latin American Restaurant,Spanish Restaurant,Deli / Bodega,Bakery,Mexican Restaurant,Fast Food Restaurant,Café,Tapas Restaurant,Restaurant
18,West Farms,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Italian Restaurant,Donut Shop,Deli / Bodega,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Breakfast Spot,Diner,Bakery
20,Melrose,Bronx,1,0,4,4,1,0,4,4,1,0,4,Sandwich Place,Pizza Place,Mexican Restaurant,Donut Shop,Diner,Fried Chicken Joint,Deli / Bodega,Caribbean Restaurant,Fast Food Restaurant,Latin American Restaurant
23,Longwood,Bronx,1,0,4,4,1,0,4,4,1,0,4,Pizza Place,Mexican Restaurant,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Donut Shop,Spanish Restaurant,Diner,Latin American Restaurant,Chinese Restaurant
